In [ ]:
%%sh

# The Version of MxNet for the docker
mxnet_version="0.12.1"

#Choose a platform. cpu or gpu
mxnet_platform="cpu"

#Choose python version py2 or py3
mxnet_python="py2" 

# Create the SageMaker MXNet Container Python package.
python setup.py sdist

#. Copy your Python package to "final" Dockerfile directory that you are building.
cp dist/sagemaker_mxnet_container-1.0.0.tar.gz docker/${mxnet_version}/final/${mxnet_python}



#Build base images first
cd docker/${mxnet_version}/base
docker build -t mxnet-base:${mxnet_version}-${mxnet_platform}-${mxnet_python} -f Dockerfile.${mxnet_platform} .

#Build base final image
cd ..
cd final/${mxnet_python}
docker build -t preprod-mxnet:${mxnet_version}-${mxnet_platform}-${mxnet_python} -f Dockerfile.${mxnet_platform} .

    
#Setting up to call ECR
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/preprod-mxnet:${mxnet_version}-${mxnet_platform}-${mxnet_python}"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "preprod-mxnet" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "preprod-mxnet" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)


# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker tag "preprod-mxnet:${mxnet_version}-${mxnet_platform}-${mxnet_python}" ${fullname}
docker push ${fullname}

In [ ]:
%%sh

# The Version of MxNet for the docker
mxnet_version="1.1.0"

#Choose a platform. cpu or gpu
mxnet_platform="cpu"

#Choose python version py2 or py3
mxnet_python="py2" 

# Create the SageMaker MXNet Container Python package.
python setup.py sdist

#. Copy your Python package to "final" Dockerfile directory that you are building.
cp dist/sagemaker_mxnet_container-1.0.0.tar.gz docker/${mxnet_version}/final

cd docker/${mxnet_version}/final
docker build -t preprod-mxnet:${mxnet_version}-${mxnet_platform}-${mxnet_python} --build-arg py_version=2 --build-arg framework_installable=mxnet-${mxnet_version}-py2.py3-none-manylinux1_x86_64.whl -f Dockerfile.${mxnet_platform} .
    
#Setting up to call ECR
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/preprod-mxnet:${mxnet_version}-${mxnet_platform}-${mxnet_python}"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "preprod-mxnet" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "preprod-mxnet" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)


# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker tag "preprod-mxnet:${mxnet_version}-${mxnet_platform}-${mxnet_python}" ${fullname}
docker push ${fullname}